In [1]:
import pandas as pd
import numpy as np
import csv
import os
HITTER_FILE = 'mlb_data/hittersByGame.csv'
PITCHER_FILE = 'mlb_data/pitchersByGame.csv'
GAME_FILE = 'mlb_data/games.csv'

In [2]:
# mapping between game ID and season
map_gameid_season = dict()
raw_data_game = pd.read_csv(GAME_FILE)
for i in range(len(raw_data_game)):
    map_gameid_season[raw_data_game['Game'][i]] = raw_data_game['Date'][i][0:4]

In [7]:
# Accumulate pitcher data over each season
pitcher_data = {'2016': dict(), '2017': dict(), '2018': dict(), '2019': dict(), '2020': dict(), '2021': dict()}
raw_data_pitcher = pd.read_csv(PITCHER_FILE)
for i in range(len(raw_data_pitcher)):
    pitcher = raw_data_pitcher['Pitchers'][i]
    if pitcher == 'TEAM':
        continue
    season = map_gameid_season[raw_data_pitcher['Game'][i]]
    team = raw_data_pitcher['Team'][i]
    if team not in pitcher_data[season]:
        pitcher_data[season][team] = dict()
    if pitcher not in pitcher_data[season][team]:
        pitcher_data[season][team][pitcher] = {'IP': 0, 'H': 0, 'R': 0, 'ER': 0, 'BB': 0, 'K': 0, 'HR': 0}
    for cat in pitcher_data[season][team][pitcher]:
        pitcher_data[season][team][pitcher][cat] += float(raw_data_pitcher[cat][i])

<ipython-input-7-e9b9e88e52b3>:3: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_pitcher = pd.read_csv(PITCHER_FILE)


In [40]:
# Store the data into csv
csv_columns = ['Season', 'Teams', 'Pitcher', 'IP', 'H', 'R', 'ER', 'BB', 'K', 'HR', 'ERA']
dict_data = list()
for season in pitcher_data:
    for team in pitcher_data[season]:
        for pitcher in pitcher_data[season][team]:
            t = dict()
            t['Season'] = season
            t['Teams'] = team
            t['Pitcher'] = pitcher
            t['IP'] = int(pitcher_data[season][team][pitcher]['IP'])
            t['H'] = int(pitcher_data[season][team][pitcher]['H'])
            t['R'] = int(pitcher_data[season][team][pitcher]['R'])
            t['ER'] = int(pitcher_data[season][team][pitcher]['ER'])
            t['K'] = int(pitcher_data[season][team][pitcher]['K'])
            t['HR'] = int(pitcher_data[season][team][pitcher]['HR'])
            t['BB'] = int(pitcher_data[season][team][pitcher]['BB'])
            if t['IP'] != 0:
                t['ERA'] = "{0:.2f}".format(9 * t['ER'] / t['IP'])
            else:
                t['ERA'] = 0
            dict_data.append(t)
csv_file = "pitcher_data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for d in dict_data:
            writer.writerow(d)
except IOError:
    print("I/O error")

In [30]:
# Accumulate pitcher data over each season
hitter_data = {'2016': dict(), '2017': dict(), '2018': dict(), '2019': dict(), '2020': dict(), '2021': dict()}
raw_data_hitter = pd.read_csv(HITTER_FILE)
for i in range(len(raw_data_hitter)):
    hitter = raw_data_hitter['Hitters'][i]
    if hitter == 'TEAM' or raw_data_hitter['H-AB'][i] == '-----':
        continue
    season = map_gameid_season[raw_data_hitter['Game'][i]]
    team = raw_data_hitter['Team'][i]
    if team not in hitter_data[season]:
        hitter_data[season][team] = dict()
    if hitter not in hitter_data[season][team]:
        hitter_data[season][team][hitter] = {'AB': 0, 'R': 0, 'H': 0, 'RBI': 0, 'BB': 0, 'K': 0, 'SLG': 0}
    for cat in hitter_data[season][team][hitter]:
        if cat == 'SLG':
            if raw_data_hitter['SLG'][i] != '--':
                hitter_data[season][team][hitter]['SLG'] += float(raw_data_hitter['SLG'][i]) * float(raw_data_hitter['AB'][i])
        else:
            hitter_data[season][team][hitter][cat] += float(raw_data_hitter[cat][i])

<ipython-input-30-1498af3c546a>:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_hitter = pd.read_csv(HITTER_FILE)


In [39]:
# Store the data into csv
csv_columns_hitter = ['Season', 'Teams', 'Hitter', 'AB', 'R', 'H', 'RBI', 'BB', 'K', 'SLG', 'AVG', 'OBP']
dict_data_hitter = list()
for season in hitter_data:
    for team in hitter_data[season]:
        for hitter in hitter_data[season][team]:
            t = dict()
            t['Season'] = season
            t['Teams'] = team
            t['Hitter'] = hitter
            t['AB'] = int(hitter_data[season][team][hitter]['AB'])
            t['R'] = int(hitter_data[season][team][hitter]['R'])
            t['H'] = int(hitter_data[season][team][hitter]['H'])
            t['RBI'] = int(hitter_data[season][team][hitter]['RBI'])
            t['BB'] = int(hitter_data[season][team][hitter]['BB'])
            t['K'] = int(hitter_data[season][team][hitter]['K'])
            if hitter_data[season][team][hitter]['AB'] != 0:
                t['SLG'] = "{0:.2f}".format(hitter_data[season][team][hitter]['SLG'] / hitter_data[season][team][hitter]['AB'])
                t['AVG'] = "{0:.2f}".format(t['H'] / t['AB'])
                t['OBP'] = "{0:.2f}".format((t['H'] + t['R']) / (t['AB'] + t['R']))
            else:
                t['SLG'] = 0
                t['AVG'] = 0
                t['OBP'] = 0
            dict_data_hitter.append(t)
csv_file = "hitter_data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns_hitter)
        writer.writeheader()
        for d in dict_data_hitter:
            writer.writerow(d)
except IOError:
    print("I/O error")